In [46]:
import pandas as pd
import numpy as np
import random

In [2]:

N_client = 10
N_product = 15

client_1 = {i: f'client_{i}' for i in range(N_client)}
client_2 = {i: f'client_{i}' for i in range(3, N_client+3)}

data_product = {i: f'product_{i}' for i in range(N_product )}
price_1 = {i: (i+1)*1000 for i in range(15)}
price_2 = {i: (i+1)*900 for i in range(15)}

client_list_1 = np.random.randint(0, N_client, 100)
client_list_2 = np.random.randint(3, N_client+3, 100)

product_list = np.random.randint(0, N_product , 100)

shop_1 = pd.DataFrame({'client': client_list_1,
                       'product': product_list,
                       'price': product_list}).replace({'client': client_1, 'product': data_product, 'price': price_1})
shop_2 = pd.DataFrame({'client': client_list_2,
                       'product': product_list,
                       'price': product_list}).replace({'client': client_2, 'product': data_product, 'price': price_2})
product = pd.DataFrame({'product': data_product.values(),
                        'color': np.random.choice(['black', 'red', 'green', 'white', 'blue'], 15),
                        'weight': np.random.randint(10, 20, 15)*10})

In [4]:
product.head()

,product,color,weight
0,product_0,red,110
1,product_1,blue,190
2,product_2,blue,170
3,product_3,red,110
4,product_4,black,120


In [5]:
shop_1.merge(product,how='left')

,client,product,price,color,weight
0,client_8,product_4,5000,black,120
1,client_8,product_10,11000,green,110
2,client_1,product_3,4000,red,110
3,client_1,product_9,10000,green,170
4,client_4,product_10,11000,green,110
...,...,...,...,...,...
95,client_0,product_12,13000,blue,140
96,client_1,product_11,12000,white,160
97,client_7,product_12,13000,blue,140
98,client_7,product_10,11000,green,110


In [11]:
shop_2.groupby('product')['price'].value_counts()

product     price
product_0   900       4
product_1   1800      8
product_10  9900     10
product_11  10800     9
product_12  11700     9
product_13  12600     4
product_14  13500     4
product_2   2700      5
product_3   3600     11
product_4   4500      6
product_5   5400      4
product_6   6300      4
product_7   7200      7
product_8   8100      7
product_9   9000      8
Name: count, dtype: int64

In [18]:
shop_2.groupby('product')['price'].first()


product
product_0       900
product_1      1800
product_10     9900
product_11    10800
product_12    11700
product_13    12600
product_14    13500
product_2      2700
product_3      3600
product_4      4500
product_5      5400
product_6      6300
product_7      7200
product_8      8100
product_9      9000
Name: price, dtype: int32

In [19]:
group= shop_2.groupby('product')['price'].first().reset_index()
new_df = shop_1.merge(group, on='product', how='left')
new_df['price_y'] = new_df['price_y'].astype(float)
new_df

,client,product,price_x,price_y
0,client_8,product_4,5000,4500.0
1,client_8,product_10,11000,9900.0
2,client_1,product_3,4000,3600.0
3,client_1,product_9,10000,9000.0
4,client_4,product_10,11000,9900.0
...,...,...,...,...
95,client_0,product_12,13000,11700.0
96,client_1,product_11,12000,10800.0
97,client_7,product_12,13000,11700.0
98,client_7,product_10,11000,9900.0


In [26]:
new_df = shop_1.merge(shop_2[['product', 'price']], on='product', how='left', suffixes=('_shop_1', '_shop_2'))
new_df['price_shop_1'] = new_df['price_shop_1'].astype(float)
new_df['price_shop_2'] = new_df['price_shop_2'].astype(float)
del new_df['client']
new_df

,product,price_shop_1,price_shop_2
0,product_4,5000.0,4500.0
1,product_4,5000.0,4500.0
2,product_4,5000.0,4500.0
3,product_4,5000.0,4500.0
4,product_4,5000.0,4500.0
...,...,...,...
745,product_11,12000.0,10800.0
746,product_11,12000.0,10800.0
747,product_11,12000.0,10800.0
748,product_11,12000.0,10800.0


In [42]:


group_1 = shop_1.groupby('client')['price'].sum()
group_2 = shop_2.groupby('client')['price'].sum()
new_df = pd.merge(group_1, group_2, on='client', how='outer').reset_index()
new_df.rename(columns={'price_x': 'sum_shop_1',
                       'price_y': 'sum_shop_2'}, inplace=True)
new_df

,client,sum_shop_1,sum_shop_2
0,client_0,73000.0,NaN
1,client_1,109000.0,NaN
2,client_10,NaN,54000.0
3,client_11,NaN,67500.0
4,client_12,NaN,89100.0
5,client_2,83000.0,NaN
6,client_3,76000.0,44100.0
7,client_4,61000.0,108900.0
8,client_5,77000.0,68400.0
9,client_6,49000.0,91800.0


In [44]:
shop_2_prices = shop_2.groupby('product')['price'].mean().reset_index()


client_1_data = shop_1[shop_1['client'] == 'client_1']
merged_data = pd.merge(client_1_data, shop_2_prices, on='product', how='left', suffixes=('_shop1', '_shop2'))


merged_data['savings'] = merged_data['price_shop1'] - merged_data['price_shop2']
sale = merged_data['savings'].sum()
sale

10900.0

In [48]:
df = pd.DataFrame([random.randint(100, 999) for i in range(49)], index=[i % 7 for i in range(49)], columns=['sum'])
print(df)

   sum
0  669
1  518
2  743
3  650
4  871
5  894
6  243
0  288
1  439
2  418
3  756
4  648
5  632
6  575
0  727
1  368
2  608
3  227
4  508
5  410
6  901
0  620
1  348
2  833
3  237
4  748
5  146
6  750
0  152
1  962
2  337
3  357
4  392
5  904
6  392
0  919
1  779
2  923
3  170
4  696
5  665
6  980
0  216
1  159
2  871
3  442
4  537
5  617
6  546


In [56]:
new_df = df.rolling(window=2, min_periods=1).mean()
new_df = df.rolling(window=2, min_periods=1).mean()

new_df = new_df[new_df.index == 6]

new_df

,sum
6,568.5
6,603.5
6,655.5
6,448.0
6,648.0
6,822.5
6,581.5


In [2]:
import pandas as pd


,user,date
0,user_50,2023-01-01
1,user_28,2023-01-01
2,user_27,2023-01-01
3,user_50,2023-01-01
4,user_3,2023-01-01


In [32]:
df = pd.read_csv('data/user_visit.csv')
df['date'] = pd.to_datetime(df['date'])
dau = df.groupby('date')['user'].unique().reset_index()
dau.head()

,date,user
0,2023-01-01,"[user_50, user_28, user_27, user_3, user_13, u..."
1,2023-01-02,"[user_3, user_13, user_16, user_15, user_21, u..."
2,2023-01-03,"[user_0, user_17, user_15, user_23, user_24]"
3,2023-01-04,"[user_22, user_5, user_4, user_23, user_17]"
4,2023-01-05,"[user_14, user_19, user_23, user_8, user_6, us..."


In [31]:

df['week_num'] = df['date'].dt.isocalendar().week

wau = df.groupby('week_num', sort= False)['user'].nunique().reset_index()
wau.set_index('week_num', inplace=True)
wau.rename(columns={'user': 'WAU'}, inplace=True)

wau.head()

,WAU
week_num,
52,7
1,24
2,21
3,26
4,24
